In [2]:
# Import necessary libraries

import pandas as pd
import numpy as np
from scipy import stats

# Load science-ready dataset
df = pd.read_csv("exoplanets_science_ready.csv")

In [4]:
# -----------------------------
# Recreate analytical strata
# -----------------------------

HZ_LOWER = 0.25
HZ_UPPER = 1.75
ROCKY_THRESHOLD = 1.6

df = df.copy()

df["in_habitable_zone"] = (
    (df["insolation_flux_earth_flux"] >= HZ_LOWER) &
    (df["insolation_flux_earth_flux"] <= HZ_UPPER)
)

df["composition"] = np.where(
    df["planet_radius_earth_radius"] < ROCKY_THRESHOLD,
    "Rocky",
    "Gaseous"
)

# -----------------------------
# Helper functions
# -----------------------------

def cohens_d(x, y):
    nx, ny = len(x), len(y)
    vx, vy = x.var(ddof=1), y.var(ddof=1)
    pooled_std = np.sqrt(((nx - 1) * vx + (ny - 1) * vy) / (nx + ny - 2))
    return (x.mean() - y.mean()) / pooled_std

# -----------------------------
# Effect size: Rocky planets (radius)
# -----------------------------

rocky = df[df["composition"] == "Rocky"]

rocky_in = rocky[rocky["in_habitable_zone"]]["planet_radius_earth_radius"].dropna()
rocky_out = rocky[~rocky["in_habitable_zone"]]["planet_radius_earth_radius"].dropna()

d_rocky = cohens_d(rocky_in, rocky_out)

print("Rocky Planets – Radius")
print("Cohen's d =", round(d_rocky, 4))

# -----------------------------
# Effect size: Gaseous planets (radius)
# -----------------------------

gas = df[df["composition"] == "Gaseous"]

gas_in = gas[gas["in_habitable_zone"]]["planet_radius_earth_radius"].dropna()
gas_out = gas[~gas["in_habitable_zone"]]["planet_radius_earth_radius"].dropna()

d_gas = cohens_d(gas_in, gas_out)

print("\nGaseous Planets – Radius")
print("Cohen's d =", round(d_gas, 4))

# -----------------------------
# Confidence Intervals (Bootstrap)
# -----------------------------

def bootstrap_ci(x, y, n_boot=5000, alpha=0.05):
    diffs = []
    for _ in range(n_boot):
        xb = np.random.choice(x, size=len(x), replace=True)
        yb = np.random.choice(y, size=len(y), replace=True)
        diffs.append(xb.mean() - yb.mean())
    lower = np.percentile(diffs, 100 * alpha / 2)
    upper = np.percentile(diffs, 100 * (1 - alpha / 2))
    return lower, upper

ci_rocky = bootstrap_ci(rocky_in.values, rocky_out.values)
ci_gas = bootstrap_ci(gas_in.values, gas_out.values)

print("\n95% CI (Mean Radius Difference)")
print("Rocky:", tuple(round(v, 4) for v in ci_rocky))
print("Gaseous:", tuple(round(v, 4) for v in ci_gas))

Rocky Planets – Radius
Cohen's d = -0.2407

Gaseous Planets – Radius
Cohen's d = -0.5721

95% CI (Mean Radius Difference)
Rocky: (-0.1355, 0.0118)
Gaseous: (-3.6933, -2.391)
